In [59]:
import numpy as np
from PIL import Image
from torchvision.transforms import functional as F
from pathlib import Path
from natsort import natsorted
from utils import save_colored_mask

In [25]:
output_dir = Path(r'./output')
image = Path(r'D:\datasets\yq-tech\data\image\390.png')
img = Image.open(image)

In [62]:
# 设置一下调整参数和输出路径
contrast_factor = [0.6, 1.5, 2, -0.6, -1.5, -2]
operation = 'adjust_contrast'
for factor in contrast_factor:
    deal_img = F.adjust_contrast(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [74]:
brightness_factor = [0.6, 0.8, 1.2, 1.4, 1.6, 2]
operation = 'adjust_brightness'
for factor in brightness_factor:
    deal_img = F.adjust_brightness(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [83]:
saturation_factor = [0.6, 1.5, 2, -0.6, -1.5, -2]
operation = 'adjust_saturation'
for factor in saturation_factor:
    deal_img = F.adjust_saturation(img, factor)
    output_path = output_dir.joinpath(f"{operation}_{factor}.png")
    deal_img.save(output_path)

In [6]:
mask = Path(r'18_mask.png')
m = Image.open(mask)

In [46]:
target_list = natsorted(list(Path(r'D:\datasets\Massachusetts_Dataset\road mask').glob('*.png')))
mask_list = natsorted(list(Path(r'D:\datasets\Massachusetts_Dataset\palette-mask').glob('*.png')))

In [68]:
al_map = [
    (0, 0, 0),  # 黑色 未识别
    (0, 128, 0),  # 林地
    (128, 0, 0),  # 种植地
    (96, 0, 0),  # 水体
    (0, 0, 255),  # 建筑物
    (128, 0, 128),  # 道路
    (191, 0, 0),  # 硬化地表
    (128, 128, 128),  # 裸地
    (128, 128, 0),  # 草地
    (105, 105, 105)  # 浅灰色 其他
]

In [72]:
idx = 0
replace_label = 5
target_label = 1
target_mask = np.asarray(Image.open(target_list[idx]))
mask = np.asarray(Image.open(mask_list[idx])).copy()

In [73]:
# 消除mask的原目标标签
locations = mask == replace_label
mask[locations] = 0
# 写入目标mask的标签
locations = target_mask == target_label
mask[locations] = replace_label

In [74]:
save_colored_mask(mask, Path(r'D:\datasets\Massachusetts_Dataset\temp\1.png'), al_map)